<a href="https://colab.research.google.com/github/fawrama/diabetes_detecion_model/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [52]:
file_location = 'https://github.com/fawrama/diabetes_detecion_model/blob/bca7a90295c9abb20ea23bac89129b2abc0a34a2/diabetes.csv?raw=true'
all_ds = pd.read_csv(file_location)
all_ds

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [53]:
x = all_ds.iloc[:,:-1]
y = all_ds.iloc[:,-1:]

In [54]:
le =  preprocessing.LabelEncoder()
x = x.apply(le.fit_transform)

In [55]:
print(x.shape)
print('X-->: ',x)
print('Y-->: ',y)

(768, 8)
X-->:       Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin  BMI  \
0              6       86             22             28        0  123   
1              1       23             19             22        0   62   
2              8      121             17              0        0   30   
3              1       27             19             16       62   77   
4              0       75              4             28      102  209   
..           ...      ...            ...            ...      ...  ...   
763           10       39             25             41      108  118   
764            2       60             21             20        0  155   
765            5       59             22             16       71   58   
766            1       64             14              0        0   95   
767            1       31             21             24        0   98   

     DiabetesPedigreeFunction  Age  
0                         350   29  
1                         196   1

In [56]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)

In [57]:
x_train

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
62,5,1,16,0,0,47,331,15
359,1,132,25,29,135,152,442,8
481,0,61,32,30,0,139,76,8
713,0,72,13,13,150,60,197,0
403,9,10,26,18,0,107,144,17
...,...,...,...,...,...,...,...,...
392,1,69,17,7,169,34,219,0
50,1,41,27,4,51,5,280,1
385,1,57,10,6,22,21,83,3
576,6,46,5,13,83,37,423,14


In [58]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
scaler.fit(x_test)

StandardScaler()

In [59]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(10,10,10),max_iter=1000)
y_train=y_train.astype('int')
mlp.fit(x_train,y_train.values.ravel())

MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)

In [60]:
pred = mlp.predict(x_test)

In [61]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(pred,y_test))
print(classification_report(y_test,pred))


[[109  38]
 [ 35  49]]
              precision    recall  f1-score   support

           0       0.74      0.76      0.75       144
           1       0.58      0.56      0.57        87

    accuracy                           0.68       231
   macro avg       0.66      0.66      0.66       231
weighted avg       0.68      0.68      0.68       231



In [62]:
pip install keras-tuner --upgrade

In [63]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [64]:
def build_model(hp):
  model = keras.Sequential()
  for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=64),
                               activation='relu'))
  #one output layer as our prediction would either be 1 or 0
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4, 1e-5])),
        loss='mean_absolute_error',
        metrics=['accuracy'])
  return model

In [65]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=6,
    executions_per_trial=3,
    project_name='Diabetes prediction')

INFO:tensorflow:Reloading Oracle from existing project ./Diabetes prediction/oracle.json
INFO:tensorflow:Reloading Tuner from ./Diabetes prediction/tuner0.json


In [66]:
tuner.search_space_summary()

Search space summary
Default search space size: 21
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 64, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 64, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001, 1e-05], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 64, 'sampling': None}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 64, 'sampling': None}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 64, 'sampling': None}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 64, 'sampling': None}
units_6 (Int)
{'default': No

In [67]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.30)

In [68]:
tuner.search(x_train, y_train, epochs=4, validation_data=(x_test, y_test))

INFO:tensorflow:Oracle triggered exit


In [69]:
tuner.results_summary()

Results summary
Results in ./Diabetes prediction
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 19
units_0: 160
units_1: 288
learning_rate: 0.001
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
units_15: 32
units_16: 32
units_17: 32
units_18: 32
Score: 0.6969696879386902
Trial summary
Hyperparameters:
num_layers: 17
units_0: 480
units_1: 96
learning_rate: 0.0001
units_2: 160
units_3: 96
units_4: 96
units_5: 224
units_6: 480
units_7: 96
units_8: 352
units_9: 160
units_10: 288
units_11: 352
units_12: 224
units_13: 416
units_14: 96
units_15: 416
units_16: 288
units_17: 480
units_18: 352
Score: 0.6969696879386902
Trial summary
Hyperparameters:
num_layers: 5
units_0: 416
units_1: 32
learning_rate: 0.0001
units_2: 288
units_3: 160
units_4: 480
units_5: 480
units_6: 288
units_7: 96
units_8: 224
units_9: 160
units_10: 480
units_11: 224
units_12: 288
units_13: 480
